# Regression Week 5: LASSO (coordinate descent)

In this notebook, you will implement your very own LASSO solver via coordinate descent. You will:
* Write a function to normalize features
* Implement coordinate descent for LASSO
* Explore effects of L1 penalty

# Fire up graphlab create

Make sure you have the latest version of graphlab (>= 1.7)

In [1]:
import graphlab

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [3]:
sales = graphlab.SFrame('data/kc_house_data.gl/')
# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to int, before using it below
sales['floors'] = sales['floors'].astype(int) 

If we want to do any "feature engineering" like creating new features or adjusting existing ones we should do this directly using the SFrames as seen in the first notebook of Week 2. For this notebook, however, we will work with the existing features.

# Import useful functions from previous notebook

As in Week 2, we convert the SFrame into a 2D Numpy array. Copy and paste `get_num_data()` from the second notebook of Week 2.

In [4]:
import numpy as np # note this allows us to refer to numpy as np instead 

In [5]:
def get_numpy_data(data_sframe, features, output):
    data_sframe['constant'] = 1
    features = ['constant'] + features
    features_sframe = data_sframe[features]
    
    features_matrix = features_sframe.to_numpy()
    
    output_sarray = data_sframe[output]
    output_array = output_sarray.to_numpy()
    
    return(features_matrix, output_array)

Also, copy and paste the `predict_output()` function to compute the predictions for an entire matrix of features given the matrix and the weights:

In [6]:
def predict_output(feature_matrix, weights):
    predictions = np.dot(feature_matrix, weights)
    return(predictions)

# Normalize features
In the house dataset, features vary wildly in their relative magnitude: `sqft_living` is very large overall compared to `bedrooms`, for instance. As a result, weight for `sqft_living` would be much smaller than weight for `bedrooms`. This is problematic because "small" weights are dropped first as `l1_penalty` goes up. 

To give equal considerations for all features, we need to **normalize features** as discussed in the lectures: we divide each feature by its 2-norm so that the transformed feature has norm 1.

Let's see how we can do this normalization easily with Numpy: let us first consider a small matrix.

In [7]:
X = np.array([[3.,5.,8.],[4.,12.,15.]])
print X

[[ 3.  5.  8.]
 [ 4. 12. 15.]]


Numpy provides a shorthand for computing 2-norms of each column:

In [8]:
norms = np.linalg.norm(X, axis=0) # gives [norm(X[:,0]), norm(X[:,1]), norm(X[:,2])]
print norms

[ 5. 13. 17.]


To normalize, apply element-wise division:

In [9]:
print X / norms # gives [X[:,0]/norm(X[:,0]), X[:,1]/norm(X[:,1]), X[:,2]/norm(X[:,2])]

[[0.6        0.38461538 0.47058824]
 [0.8        0.92307692 0.88235294]]


Using the shorthand we just covered, write a short function called `normalize_features(feature_matrix)`, which normalizes columns of a given feature matrix. The function should return a pair `(normalized_features, norms)`, where the second item contains the norms of original features. As discussed in the lectures, we will use these norms to normalize the test data in the same way as we normalized the training data. 

In [12]:
def normalize_features(feature_matrix):
    norms = np.linalg.norm(feature_matrix, axis=0)
    return (feature_matrix/norms, norms)

To test the function, run the following:

In [13]:
features, norms = normalize_features(np.array([[3.,6.,9.],[4.,8.,12.]]))
print features
# should print
# [[ 0.6  0.6  0.6]
#  [ 0.8  0.8  0.8]]
print norms
# should print
# [5.  10.  15.]

[[0.6 0.6 0.6]
 [0.8 0.8 0.8]]
[ 5. 10. 15.]


# Implementing Coordinate Descent with normalized features

We seek to obtain a sparse set of weights by minimizing the LASSO cost function
```
SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|).
```
(By convention, we do not include `w[0]` in the L1 penalty term. We never want to push the intercept to zero.)

The absolute value sign makes the cost function non-differentiable, so simple gradient descent is not viable (you would need to implement a method called subgradient descent). Instead, we will use **coordinate descent**: at each iteration, we will fix all weights but weight `i` and find the value of weight `i` that minimizes the objective. That is, we look for
```
argmin_{w[i]} [ SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|) ]
```
where all weights other than `w[i]` are held to be constant. We will optimize one `w[i]` at a time, circling through the weights multiple times.  
  1. Pick a coordinate `i`
  2. Compute `w[i]` that minimizes the cost function `SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|)`
  3. Repeat Steps 1 and 2 for all coordinates, multiple times

For this notebook, we use **cyclical coordinate descent with normalized features**, where we cycle through coordinates 0 to (d-1) in order, and assume the features were normalized as discussed above. The formula for optimizing each coordinate is as follows:
```
       ┌ (ro[i] + lambda/2)     if ro[i] < -lambda/2
w[i] = ├ 0                      if -lambda/2 <= ro[i] <= lambda/2
       └ (ro[i] - lambda/2)     if ro[i] > lambda/2
```
where
```
ro[i] = SUM[ [feature_i]*(output - prediction + w[i]*[feature_i]) ].
```

Note that we do not regularize the weight of the constant feature (intercept) `w[0]`, so, for this weight, the update is simply:
```
w[0] = ro[i]
```

## Effect of L1 penalty

Let us consider a simple model with 2 features:

In [14]:
simple_features = ['sqft_living', 'bedrooms']
my_output = 'price'
(simple_feature_matrix, output) = get_numpy_data(sales, simple_features, my_output)

Don't forget to normalize features:

In [15]:
simple_feature_matrix, norms = normalize_features(simple_feature_matrix)

We assign some random set of initial weights and inspect the values of `ro[i]`:

In [16]:
weights = np.array([1., 4., 1.])

Use `predict_output()` to make predictions on this data.

In [17]:
prediction = predict_output(simple_feature_matrix, weights)

Compute the values of `ro[i]` for each feature in this simple model, using the formula given above, using the formula:
```
ro[i] = SUM[ [feature_i]*(output - prediction + w[i]*[feature_i]) ]
```

*Hint: You can get a Numpy vector for feature_i using:*
```
simple_feature_matrix[:,i]
```

In [31]:
ro_is = []
for i in range(1, 3):
    feature_i = simple_feature_matrix[:,i]
    diff = output - prediction + (weights[i] * feature_i)
    ro_i = sum(feature_i * diff)
    
    ro_is.append((i, ro_i))

In [32]:
ro_is

[(1, 87939470.7729908), (2, 80966698.67596525)]

***QUIZ QUESTION***

Recall that, whenever `ro[i]` falls between `-l1_penalty/2` and `l1_penalty/2`, the corresponding weight `w[i]` is sent to zero. Now suppose we were to take one step of coordinate descent on either feature 1 or feature 2. What range of values of `l1_penalty` **would not** set `w[1]` zero, but **would** set `w[2]` to zero, if we were to take a step in that coordinate? 

***QUIZ QUESTION***

What range of values of `l1_penalty` would set **both** `w[1]` and `w[2]` to zero, if we were to take a step in that coordinate? 

So we can say that `ro[i]` quantifies the significance of the i-th feature: the larger `ro[i]` is, the more likely it is for the i-th feature to be retained.

## Single Coordinate Descent Step

Using the formula above, implement coordinate descent that minimizes the cost function over a single feature i. Note that the intercept (weight 0) is not regularized. The function should accept feature matrix, output, current weights, l1 penalty, and index of feature to optimize over. The function should return new weight for feature i.

In [38]:
def lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty):
    # compute prediction
    prediction = predict_output(feature_matrix, weights)
    # compute ro[i] = SUM[ [feature_i]*(output - prediction + weight[i]*[feature_i]) ]
    feature_i = feature_matrix[:,i]
    ro_i = sum(feature_i * (output - prediction + feature_i * weights[i]))

    if i == 0: # intercept -- do not regularize
        new_weight_i = ro_i 
    elif ro_i < -l1_penalty/2.:
        new_weight_i = ro_i + l1_penalty/2
    elif ro_i > l1_penalty/2.:
        new_weight_i = ro_i - l1_penalty/2
    else:
        new_weight_i = 0.
    
    return new_weight_i

To test the function, run the following cell:

In [39]:
# should print 0.425558846691
import math
print lasso_coordinate_descent_step(1, np.array([[3./math.sqrt(13),1./math.sqrt(10)],[2./math.sqrt(13),3./math.sqrt(10)]]), 
                                   np.array([1., 1.]), np.array([1., 4.]), 0.1)

0.4255588466910251


## Cyclical coordinate descent 

Now that we have a function that optimizes the cost function over a single coordinate, let us implement cyclical coordinate descent where we optimize coordinates 0, 1, ..., (d-1) in order and repeat.

When do we know to stop? Each time we scan all the coordinates (features) once, we measure the change in weight for each coordinate. If no coordinate changes by more than a specified threshold, we stop.

For each iteration:
1. As you loop over features in order and perform coordinate descent, measure how much each coordinate changes.
2. After the loop, if the maximum change across all coordinates is falls below the tolerance, stop. Otherwise, go back to step 1.

Return weights

**IMPORTANT: when computing a new weight for coordinate i, make sure to incorporate the new weights for coordinates 0, 1, ..., i-1. One good way is to update your weights variable in-place. See following pseudocode for illustration.**
```
for i in range(len(weights)):
    old_weights_i = weights[i] # remember old value of weight[i], as it will be overwritten
    # the following line uses new values for weight[0], weight[1], ..., weight[i-1]
    #     and old values for weight[i], ..., weight[d-1]
    weights[i] = lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty)
    
    # use old_weights_i to compute change in coordinate
    ...
```

In [49]:
l1 = [1, 2, 3]
l1[2] = 4
l1

[1, 2, 4]

In [115]:
def lasso_cyclical_coordinate_descent(feature_matrix, output, weights, l1_penalty, tolerance):
    print("Weights:" + str(weights))
    coordinate_changes = []
    for i in range(len(initial_weights)):
        new_weight_i = lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty)
        coordinate_changes.append(abs(weights[i] - new_weight_i))
        weights[i] = new_weight_i
    
    print("Coordinate Changes: " + str(coordinate_changes))
    if max(coordinate_changes) >= tolerance:
        return lasso_cyclical_coordinate_descent(feature_matrix, output, weights, l1_penalty, tolerance)
    else:
        return weights

Using the following parameters, learn the weights on the sales dataset. 

In [77]:
simple_features = ['sqft_living', 'bedrooms']
my_output = 'price'
initial_weights = np.zeros(3)
l1_penalty = 1e7
tolerance = 1.0

In [50]:
1e7 / 2

5000000.0

First create a normalized version of the feature matrix, `normalized_simple_feature_matrix`.

In [78]:
(simple_feature_matrix, output) = get_numpy_data(sales, simple_features, my_output)
(normalized_simple_feature_matrix, simple_norms) = normalize_features(simple_feature_matrix) # normalize features

Then, run your implementation of LASSO coordinate descent:

In [79]:
weights = lasso_cyclical_coordinate_descent(normalized_simple_feature_matrix, output,
                                            initial_weights, l1_penalty, tolerance)

Weights:[0. 0. 0.]
Coordinate Changes: [79400304.65805142, 10305258.636020651, 299724.11449879874]
Weights:[79400304.65805142 10305258.63602065  -299724.1144988 ]
Coordinate Changes: [9138168.366497353, 8642336.998722883, 299724.11449879874]
Weights:[70262136.29155406 18947595.63474353        0.        ]
Coordinate Changes: [8194809.409369849, 7213612.451021705, 0.0]
Weights:[62067326.88218421 26161208.08576524        0.        ]
Coordinate Changes: [6598905.038273573, 6036579.868936997, 0.0]
Weights:[55468421.84391064 32197787.95470224        0.        ]
Coordinate Changes: [5522173.194295108, 5051601.643624671, 0.0]
Weights:[49946248.64961553 37249389.59832691        0.        ]
Coordinate Changes: [4621129.810313255, 4227340.600127637, 0.0]
Weights:[45325118.83930228 41476730.19845454        0.        ]
Coordinate Changes: [3867108.1062485054, 3537572.7957573086, 0.0]
Weights:[41458010.73305377 45014302.99421185        0.        ]
Coordinate Changes: [3236118.81060721, 2960353.20288

Coordinate Changes: [14.861288469284773, 13.594884440302849, 0.0]
Weights:[21625070.17632163 63157181.09477554        0.        ]
Coordinate Changes: [12.436397407203913, 11.376630917191505, 0.0]
Weights:[21625057.73992422 63157192.47140646        0.        ]
Coordinate Changes: [10.407172374427319, 9.52032558619976, 0.0]
Weights:[21625047.33275185 63157201.99173205        0.        ]
Coordinate Changes: [8.709051750600338, 7.966910764575005, 0.0]
Weights:[21625038.6237001  63157209.95864281        0.        ]
Coordinate Changes: [7.288011036813259, 6.666963666677475, 0.0]
Weights:[21625031.33568906 63157216.62560648        0.        ]
Coordinate Changes: [6.098838798701763, 5.579127252101898, 0.0]
Weights:[21625025.23685026 63157222.20473373        0.        ]
Coordinate Changes: [5.103702429682016, 4.668790593743324, 0.0]
Weights:[21625020.13314783 63157226.87352432        0.        ]
Coordinate Changes: [4.270939916372299, 3.9069920629262924, 0.0]
Weights:[21625015.86220792 63157230

In [80]:
print(weights)

[21624998.36636352 63157246.78545319        0.        ]


In [81]:
predictions = predict_output(normalized_simple_feature_matrix, weights)
sum((predictions - output)**2)

1630492481484499.5

In [82]:
1e15 < 1630492481484499.5 < 3e15

True

***QUIZ QUESTIONS***
1. What is the RSS of the learned model on the normalized dataset? (Hint: use the normalized feature matrix when you make predictions.)
2. Which features had weight zero at convergence?

# Evaluating LASSO fit with more features

Let us split the sales dataset into training and test sets.

In [83]:
train_data,test_data = sales.random_split(.8,seed=0)

Let us consider the following set of features.

In [84]:
all_features = ['bedrooms',
                'bathrooms',
                'sqft_living',
                'sqft_lot',
                'floors',
                'waterfront', 
                'view', 
                'condition', 
                'grade',
                'sqft_above',
                'sqft_basement',
                'yr_built', 
                'yr_renovated']

First, create a normalized feature matrix from the TRAINING data with these features.  (Make you store the norms for the normalization, since we'll use them later)

In [87]:
all_feat_matrix, output = get_numpyall_feat_matrix, output = get_numpy_data(sales, all_features, my_output)
norm_feat_matrix, norms = normalize_features(all_feat_matrix)

First, learn the weights with `l1_penalty=1e7`, on the training data. Initialize weights to all zeros, and set the `tolerance=1`.  Call resulting weights `weights1e7`, you will need them later.

In [116]:
initial_weights = np.zeros(14)
l1_penalty = 1e7
tolerance = 1.0

In [117]:
weights1e7 = lasso_cyclical_coordinate_descent(norm_feat_matrix, output,
                                            initial_weights, l1_penalty, tolerance)

Weights:[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Coordinate Changes: [79400304.65805142, 0.0, 4698536.82947805, 5776463.7871389575, 0.0, 88695.29018316232, 8144113.544721436, 8445943.304970175, 7231648.057170596, 0.0, 1349249.77559626, 2364615.543230309, 3190664.53548815, 0.0]
Weights:[79400304.65805142        0.          4698536.82947805  5776463.78713896
        0.           -88695.29018316  8144113.54472144  8445943.30497018
 -7231648.0571706         0.          1349249.77559626  2364615.54323031
 -3190664.53548815        0.        ]
Coordinate Changes: [5024423.989266425, 0.0, 2481113.245323712, 4642999.414444713, 0.0, 88695.29018316232, 3341952.8313904665, 1936828.7712633293, 1240009.33330453, 0.0, 148172.5905278679, 249081.0037214039, 983174.5864634356, 0.0]
Weights:[74375880.66878499        0.          7179650.07480176 10419463.20158367
        0.                0.          4802160.71333097 10382772.0762335
 -8471657.39047513        0.          1497422.36612413  2613696.54695

Coordinate Changes: [84461.73118113726, 0.0, 0.0, 79681.63555707783, 0.0, 0.0, 344.0436555277556, 5517.71038996242, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Weights:[25422678.02441184        0.                0.         55965572.00424241
        0.                0.          3687792.45828278  8660162.4905067
        0.                0.                0.                0.
        0.                0.        ]
Coordinate Changes: [71308.08790550008, 0.0, 0.0, 67272.37514963746, 0.0, 0.0, 290.346722971648, 4658.345149276778, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Weights:[25351369.93650634        0.                0.         56032844.37939204
        0.                0.          3688082.80500575  8655504.14535742
        0.                0.                0.                0.
        0.                0.        ]
Coordinate Changes: [60202.89413273707, 0.0, 0.0, 56795.67538076639, 0.0, 0.0, 245.10512149706483, 3932.8624292146415, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Weights:[25291167.04237361        0.          

Coordinate Changes: [874.1378983296454, 0.0, 0.0, 824.6654974892735, 0.0, 0.0, 3.558778215199709, 57.10456804931164, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Weights:[24969537.44483605        0.                0.         56393066.81766627
        0.                0.          3689637.32619147  8630560.27279033
        0.                0.                0.                0.
        0.                0.        ]
Coordinate Changes: [738.0035510025918, 0.0, 0.0, 696.2357611209154, 0.0, 0.0, 3.0045500062406063, 48.21135640144348, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Weights:[24968799.44128504        0.                0.         56393763.05342739
        0.                0.          3689640.33074148  8630512.06143393
        0.                0.                0.                0.
        0.                0.        ]
Coordinate Changes: [623.0701611749828, 0.0, 0.0, 587.8071024045348, 0.0, 0.0, 2.5366344824433327, 40.70313461124897, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Weights:[24968176.37112387        0.       

Coordinate Changes: [24.98207462951541, 0.0, 0.0, 23.568197667598724, 0.0, 0.0, 0.10170669108629227, 1.6319970581680536, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Weights:[24964934.05099885        0.                0.         56397409.67944678
        0.                0.          3689656.06746512  8630259.54813682
        0.                0.                0.                0.
        0.                0.        ]
Coordinate Changes: [21.09147708117962, 0.0, 0.0, 19.897790998220444, 0.0, 0.0, 0.08586732670664787, 1.3778371158987284, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Weights:[24964912.95952177        0.                0.         56397429.57723778
        0.                0.          3689656.15333245  8630258.17029971
        0.                0.                0.                0.
        0.                0.        ]
Coordinate Changes: [17.806784011423588, 0.0, 0.0, 16.79899785667658, 0.0, 0.0, 0.07249468564987183, 1.163258995860815, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Weights:[24964895.15273776        

In [118]:
weights1e7

array([24964803.20411231,        0.        ,        0.        ,
       56397533.12096906,        0.        ,        0.        ,
        3689656.60016713,  8630251.00033792,        0.        ,
              0.        ,        0.        ,        0.        ,
              0.        ,        0.        ])

In [96]:
all_features

['bedrooms',
 'bathrooms',
 'sqft_living',
 'sqft_lot',
 'floors',
 'waterfront',
 'view',
 'condition',
 'grade',
 'sqft_above',
 'sqft_basement',
 'yr_built',
 'yr_renovated']

***QUIZ QUESTION***

What features had non-zero weight in this case?

Next, learn the weights with `l1_penalty=1e8`, on the training data. Initialize weights to all zeros, and set the `tolerance=1`.  Call resulting weights `weights1e8`, you will need them later.

In [119]:
initial_weights = np.zeros(14)
l1_penalty = 1e8
tolerance = 1.0

In [120]:
weights1e8 = lasso_cyclical_coordinate_descent(norm_feat_matrix, output,
                                            initial_weights, l1_penalty, tolerance)

Weights:[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Coordinate Changes: [79400304.65805142, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Weights:[79400304.65805142        0.                0.                0.
        0.                0.                0.                0.
        0.                0.                0.                0.
        0.                0.        ]
Coordinate Changes: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [121]:
weights1e8

array([79400304.65805142,        0.        ,        0.        ,
              0.        ,        0.        ,        0.        ,
              0.        ,        0.        ,        0.        ,
              0.        ,        0.        ,        0.        ,
              0.        ,        0.        ])

***QUIZ QUESTION***

What features had non-zero weight in this case?

Finally, learn the weights with `l1_penalty=1e4`, on the training data. Initialize weights to all zeros, and set the `tolerance=5e5`.  Call resulting weights `weights1e4`, you will need them later.  (This case will take quite a bit longer to converge than the others above.)

In [122]:
initial_weights = np.zeros(14)
l1_penalty = 1e4
tolerance = 5e5

In [123]:
weights1e4 = lasso_cyclical_coordinate_descent(norm_feat_matrix, output,
                                            initial_weights, l1_penalty, tolerance)

Weights:[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Coordinate Changes: [79400304.65805142, 4421286.338102187, 5481209.455818298, 5844296.134806477, 1078799.1097964032, 9586861.195141092, 13578416.585554106, 12453175.592991656, 9968665.52621384, 4076915.0420512203, 7259911.294948143, 2888547.5835873354, 12629088.90865757, 3564098.9613847896]
Weights:[ 79400304.65805142   4421286.33810219   5481209.4558183
   5844296.13480648  -1078799.1097964   -9586861.19514109
  13578416.58555411  12453175.59299166  -9968665.52621384
   4076915.04205122   7259911.29494814   2888547.58358734
 -12629088.90865757   3564098.96138479]
Coordinate Changes: [772486.0038247406, 2774609.820895303, 3970085.8812352857, 3885018.4513748335, 667948.1186931252, 5318569.004229035, 5560422.338330516, 1687407.5758670643, 4235823.947720181, 3206908.5241804784, 5864734.477403789, 441341.440095223, 9318832.099838953, 150378.88581297593]
Weights:[ 78627818.65422668   7195896.15899749   9451295.33705358
   9729314.58618131 

Coordinate Changes: [109981.58996239305, 1102599.559695799, 167428.1586367339, 1264886.7571221888, 1612.9870165777393, 543155.8762873262, 9389.026607257314, 41101.86726378836, 695956.8253469616, 472317.3086337019, 638257.152007591, 100506.73041189136, 438681.6483373344, 646.4725155374035]
Weights:[ 79908067.07733364  -1764345.14844634  33386467.57585508
  39435923.91019472  -2395641.30837902 -22334643.00479102
   7225403.42339825   8998366.72443984 -24687193.58960301
  12651599.43726814  26573203.67755344   3782279.67599593
 -62975122.35673854   2837816.18017164]
Coordinate Changes: [110590.17455676198, 1071104.638365986, 100074.3733874932, 1257630.7326523587, 2365.8777408348396, 526728.356134098, 10243.751335432753, 34141.87980704941, 734717.6665251106, 468983.10797215253, 661635.5953770205, 110392.07191423886, 400963.06964161247, 2539.1826676870696]
Weights:[ 80018657.2518904   -2835449.78681232  33486541.94924257
  40693554.64284708  -2398007.18611985 -21807914.64865692
   7215159.6

Coordinate Changes: [106501.08185528219, 566454.9714768529, 303657.92506479844, 1123002.5184860304, 7459.14122293843, 268928.6423275992, 8165.227470563725, 11109.490913119167, 690142.9144274443, 168951.06136489753, 668968.6245252583, 161199.69203526294, 363605.5512994081, 8856.071264354046]
Weights:[ 81552672.47051924 -13936396.10707409  31005474.32166816
  57421816.28259485  -2484787.60462152 -16534253.15328067
   7072835.10705295   8742457.96143284 -13217610.87335158
   7634030.05325642  16268968.19588774   1602242.36731965
 -68178672.24634618   2950150.64047416]
Coordinate Changes: [105886.61280286312, 537212.0503250118, 311323.2336508855, 1109348.474832885, 7364.31117069535, 257666.22159864008, 7787.519685585052, 10988.095876293257, 671224.472607391, 145341.52858564444, 664301.9445992764, 161341.3138839947, 369949.8523667604, 8740.789113156497]
Weights:[ 81658559.08332211 -14473608.1573991   30694151.08801728
  58531164.75742774  -2492151.91579221 -16276586.93168203
   7065047.5873

Coordinate Changes: [97406.18750806153, 229140.2059299834, 315728.9586114213, 894546.4749442041, 3705.4524426558055, 162594.72943295166, 3353.752014630474, 9382.16734207794, 410095.20515642874, 128510.71326561458, 606759.4883826869, 150070.01475828968, 443314.78491942585, 6146.7635414954275]
Weights:[ 83176249.76865508 -19792358.09614618  25803170.24334639
  73444890.22561032  -2575334.54663075 -13300686.28385268
   6986895.59845341   8575131.46992688  -4671597.57223109
   7690507.45134059   6127486.74813645   -893243.83249184
 -74751319.9749907    3069179.18014565]
Coordinate Changes: [96937.6891002804, 215420.25560685247, 312078.2441502623, 881218.116904676, 3460.8452593041584, 158538.57423905283, 3189.983858381398, 9198.211214696988, 396460.06704840437, 141546.4407244185, 603617.7680738447, 149165.45161361748, 446492.4667761624, 5986.851883040741]
Weights:[ 83273187.45775536 -20007778.35175304  25491091.99919613
  74326108.34251499  -2578795.39189006 -13142147.70961362
   6983705.61

Coordinate Changes: [90484.72389096022, 54381.22848157957, 224160.19557052478, 672046.9613912553, 113.36216217186302, 102556.33838175982, 634.2321775285527, 7508.534819198772, 213353.75850906502, 309143.4059524927, 545102.8428827832, 131472.0498473947, 484525.21190032363, 3537.8830399527214]
Weights:[ 85043330.9156009  -22232431.64292621  20426153.69947774
  88848124.42859957  -2605975.01692301 -10744399.79507427
   6951095.53667268   8411655.04553338   1219869.11193634
  12405008.47714121  -5324081.09772135  -3702799.55771551
 -84154625.93347715   3162259.53959402]
Coordinate Changes: [90246.07920457423, 49543.114572089165, 219492.73145756125, 663445.6552037448, 232.1221393509768, 100153.68855571002, 559.1034668460488, 7390.018484862521, 207719.7845241744, 313868.70208442956, 542583.1532280371, 130810.01415008819, 485082.91436335444, 3458.209026860539]
Weights:[ 85133576.99480547 -22281974.7574983   20206660.96802017
  89511570.08380331  -2605742.89478366 -10644246.10651856
   6950536

Coordinate Changes: [86948.84319853783, 5193.360635850579, 141454.6508903876, 538037.9879736453, 1558.4248052090406, 62018.18453145586, 417.4851773492992, 6477.883873315528, 132825.32569320686, 367936.4059528224, 495078.17405392416, 119590.55594436266, 487347.0591227561, 2224.5703326342627]
Weights:[ 8.68118480e+07 -2.25784508e+07  1.68570953e+07  1.00760695e+08
 -2.58579481e+06 -9.14926269e+06  6.95095615e+06  8.27543969e+06
  4.53792889e+06  1.93194994e+07 -1.56928695e+07 -6.20297009e+06
 -9.39092436e+07  3.21811801e+06]
Coordinate Changes: [86829.99991330504, 6610.045036699623, 138128.95158253796, 533058.5933191031, 1594.1164231821895, 60477.79641007632, 448.1967980004847, 6458.92168899253, 130244.82197446376, 369284.4302641675, 492756.1431497857, 119063.50702750031, 486934.1805757284, 2176.725501202047]
Weights:[ 8.68986780e+07 -2.25718408e+07  1.67189664e+07  1.01293754e+08
 -2.58420069e+06 -9.08878489e+06  6.95140434e+06  8.26898077e+06
  4.66817371e+06  1.96887838e+07 -1.6185625

In [124]:
weights1e4

array([ 8.75895325e+07, -2.24790006e+07,  1.57236295e+07,  1.05393774e+08,
       -2.57043643e+06, -8.65548772e+06,  6.95591317e+06,  8.21787482e+06,
        5.62865025e+06,  2.26789845e+07, -2.00465047e+07, -7.25618289e+06,
       -9.82714585e+07,  3.23616425e+06])

***QUIZ QUESTION***

What features had non-zero weight in this case?

## Rescaling learned weights

Recall that we normalized our feature matrix, before learning the weights.  To use these weights on a test set, we must normalize the test data in the same way.

Alternatively, we can rescale the learned weights to include the normalization, so we never have to worry about normalizing the test data: 

In this case, we must scale the resulting weights so that we can make predictions with *original* features:
 1. Store the norms of the original features to a vector called `norms`:
```
features, norms = normalize_features(features)
```
 2. Run Lasso on the normalized features and obtain a `weights` vector
 3. Compute the weights for the original features by performing element-wise division, i.e.
```
weights_normalized = weights / norms
```
Now, we can apply `weights_normalized` to the test data, without normalizing it!

Create a normalized version of each of the weights learned above. (`weights1e4`, `weights1e7`, `weights1e8`).

In [127]:
norms

array([1.47013605e+02, 5.14075870e+02, 3.30872804e+02, 3.34257264e+05,
       6.48161603e+06, 2.27567572e+02, 1.27671453e+01, 1.17804923e+02,
       5.10280315e+02, 1.13885118e+03, 2.89734098e+05, 7.79090452e+04,
       2.89796744e+05, 6.03405316e+04])

In [125]:
weights1e4_n = weights1e4 / norms
weights1e7_n = weights1e7 / norms
weights1e8_n = weights1e8 / norms

In [126]:
weights1e7_n[3]

168.72492889253746

To check your results, if you call `normalized_weights1e7` the normalized version of `weights1e7`, then:
```
print normalized_weights1e7[3]
```
should return 161.31745624837794.

## Evaluating each of the learned models on the test data

Let's now evaluate the three models on the test data:

In [129]:
(test_feature_matrix, test_output) = get_numpy_data(test_data, all_features, 'price')

Compute the RSS of each of the three normalized weights on the (unnormalized) `test_feature_matrix`:

In [130]:
rss_4 = sum((predict_output(test_feature_matrix, weights1e4_n) - test_output)**2)

In [131]:
rss_7 = sum((predict_output(test_feature_matrix, weights1e7_n) - test_output)**2)

In [132]:
rss_8 = sum((predict_output(test_feature_matrix, weights1e7_n) - test_output)**2)

In [138]:
(rss_4, rss_7, rss_8)

(225535784135719.47, 270215018898671.28, 270215018898671.28)

***QUIZ QUESTION***

Which model performed best on the test data?

In [139]:
min([rss_4, rss_7, rss_8])

225535784135719.47

In [134]:
[rss for rss in [rss_4, rss_7, rss_8] if rss == max([rss_4, rss_7, rss_8])]

[270215018898671.28, 270215018898671.28]